In [1]:
using Random
using Distributions

## Funciones Auxiliares

## Funciones de Mutacion

In [3]:
# testeado
# funciona para vectores booleanos
function bit_flip_mutation(v, p)
    l = length(v)
    for i in 1:l
        if p >= rand()
            v[i] = 1-v[i]
        end
    end
    return v
end

bit_flip_mutation (generic function with 1 method)

In [4]:
# testeado
# funciona para vectores floats
function gaussian_convolution(v, p, σ2, δ)
    dist = Normal(0, σ2)
    l = length(v)
    min_v = v .- δ
    max_v = v .+ δ
    for i in 1:l
        if p >= rand()
            n = Inf
            while !(min_v[i] <= v[i]+n && v[i] + n <= max_v[i])
                n = rand(dist)
            end
            v[i] = v[i] + n
        end
    end
    return v
end

gaussian_convolution (generic function with 1 method)

## Funciones de Crossover

In [5]:
# testeado
function one_point_crossover(v, w)
    l = length(v)
    c = rand(1:l)
    for i in c:l
        # hago swap
        temp = v[i]
        v[i] = w[i]
        w[i] = temp
    end
    return v, w
end

one_point_crossover (generic function with 1 method)

In [6]:
# testeado
function two_point_crossover(v, w)
    l = length(v)
    c = rand(1:l)
    d = rand(1:l)
    if c > d
        # hago swap
        temp = c
        c = d
        d = temp
    end
    for i in c:d-1
        # hago swap
        temp = v[i]
        v[i] = w[i]
        w[i] = temp
    end
    return v, w
end

two_point_crossover (generic function with 1 method)

In [7]:
# testeado
function uniform_crossover(v, w, p)
    l = length(v)
    for i in 1:l
        if p>=rand()
            # hago swap
            temp = v[i]
            v[i] = w[i]
            w[i] = temp
        end
    end
    return v, w
end


uniform_crossover (generic function with 1 method)

Hacer crossover en la poblacion P no te deja salir de un hipercubo por lo que es necesario
realizar la mutacion para explorar otras partes del espacio. El crossover hace que se
esparzan los buildingblocks por la poblacion.
Por lo general las genes no son independientes entre si, tenerlo en cuanta a la hora de
trabajar.
Ahora vamos a mezclar no solo dos sino varios vectores a la vez

In [8]:
# testeado
function shuffle_vector(v)
    l = length(v)
    for i in 2:l
        j = rand(1:l)
        # hago swap
        temp = v[i]
        v[i] = v[j]
        v[j] = temp
    end
    return v
end

shuffle_vector (generic function with 1 method)

In [9]:
# testeado
function uniform_crossover_K_vectors(W, p) # W vectores para hacer crossover
    l = length(W[1])
    k = length(W)
    v = zeros(k)
    for i in 1:l
        if p>=rand()
            for j in 1:k
                w = W[j]
                v[j] = w[i]
            end
            # randomly shuffle v
            v = shuffle_vector(v)
            for j in 1:k
                w = W[j]
                w[i] = v[j]
                W[j] = w
            end
        end
    end
    return W
end

uniform_crossover_K_vectors (generic function with 1 method)

In [10]:
# testeado
function line_recombination(v, w, p)
    l = length(v)
    a = -p
    b = 1+p
    α = rand() * (b - a) + a
    β = rand() * (b - a) + a
    for i in 1:l
        t = α*v[i] + (1-α)*w[i]
        s = β*w[i] + (1-β)*v[i]
        if true #in_bounds(t) && in_bounds(s)
            v[i] = t
            w[i] = s
        end
    end
    return v, w
end

line_recombination (generic function with 1 method)

In [11]:
# testeado
function intermediate_recombination(v, w, p)
    l = length(v)
    a = -p
    b = 1+p
    for i in 1:l
        α = rand() * (b - a) + a
        β = rand() * (b - a) + a
        t = α*v[i] + (1-α)*w[i]
        s = β*w[i] + (1-β)*v[i]
        temp = true
        while temp  #!(in_bounds(t) && in_bounds(s))
            α = rand() * (b - a) + a
            β = rand() * (b - a) + a
            t = α*v[i] + (1-α)*w[i]
            s = β*w[i] + (1-β)*v[i]
            temp = false
        end
        v[i] = t
        w[i] = s
    end
    return v, w
end

intermediate_recombination (generic function with 1 method)

## Funciones de seleccion

In [12]:
# testeado
""" precalculo que solo hace falta hacerlo una vez por generacion
l = length(f)
if f == ones(l)
    f = ones(l)
end
for i in 2:l # convierto f en un vector distribucional
    f[i] += f[i-1]
end"""
function fitness_proportionate_selection(P, f) # necesito que f>=0
    l = length(f)
    n = rand() * f[l]
    for i in 2:l
        if f[i-1] < n && n <= f[i]
            return P[i]
        end
    end
    return P[1]
end

fitness_proportionate_selection

In [61]:
# testeado
""" precalculo que solo hace falta hacerlo una vez cada n individuos seleccionados
l = length(f)
if f == ones(l)
    f = ones(l)
end
for i in 2:l # convierto f en un vector distribucional
    f[i] += f[i-1]
end
value = rand() * f[l]/n
i = 1
"""
function stochastic_universal_sampling(P, f, n, value, i)
    while f[i] < value
        i += 1
    end
    value += f[l]/n
    return P[i], value, i
end


stochastic_universal_sampling (generic function with 1 method)

In [66]:
v1 = 1 .* ones(2)
v2 = 2 .* ones(2)
v3 = 3 .* ones(2)
v4 = 4 .* ones(2)
v5 = 5 .* ones(2)
P = [v1,v2,v3,v4,v5]
f = [1,2,3,4,5]
n = 5

# precalculo que solo hace falta hacerlo una vez cada n individuos seleccionados
l = length(f)
if f == ones(l)
    f = ones(l)
end
for i in 2:l # convierto f en un vector distribucional
    f[i] += f[i-1]
end
value = rand() * f[l]/n
i = 1

for _ in 1:n
    v, value, i = stochastic_universal_sampling(P, f, n, value, i)    
    println(v)
    println(value)
    println(i)
end

5

In [41]:
#testeado
function tournament_selecction(F, P, t)
    best = rand(P)
    for i in 2:t
        next = rand(P)
        if F(next) > F(best)
            best = next
        end
    end
    return best
end

tournament_selecction (generic function with 2 methods)

## Algoritmos evolutivos

In [13]:
# (μ,λ) strategy con n genes
# F mi funcion, μ la cantidad que sobreviven de cada generacion, λ el tamaño de mi poblacion, n la cantidad de genes
function GA_1(F, μ, λ, p, σ2, δ, n; max_iter = 10000, ϵ = 1e-8)
    P = [rand(n) for i in 1:λ]
    best = P[1]
    valor_best = F(best)
    iteracion = 1
    variacion = 1
    while (iteracion <= max_iter) && variacion > ϵ
        variacion = valor_best
        for p in P
            valor_p = F(p)
            if valor_p > valor_best
                best = p
                valor_best = valor_p
            end
        end
        variacion = abs(variacion-valor_best)
        Q = sort(P, by=F, rev=true)[1:μ]
        P = []
        for q in Q
            for i in 1:(λ/μ)
                push!(P,gaussian_convolution(q, p, σ2, δ))
            end
        end
        iteracion += 1
    end
    return valor_best, best, iteracion, variacion
end

GA_1 (generic function with 1 method)

In [14]:
# test del algoritmo de arriba
F(x) = 5sin(x[1]) + 5cos(x[2]) - 0.1(x[1])^2 - 0.1(x[2])^2 
μ = 5
λ = 25
p = 0.3
σ2 = 0.5
δ = 0.1
n = 2
GA_1(F, μ, λ, p, σ2, δ, n)

(8.76635467365866, [0.8152105402961077, -0.25849279766506605], 3, 0.0)

In [15]:
# μ+λ strategy con n genes
# F mi funcion, μ la cantidad que sobreviven de cada generacion, λ el tamaño de mi poblacion, n la cantidad de genes
function GA_2(F, μ, λ, p, σ2, δ, n; max_iter = 10000, ϵ = 1e-8)
    P = [rand(n) for i in 1:λ]
    best = P[1]
    valor_best = F(best)
    iteracion = 1
    variacion = 1
    while (iteracion <= max_iter) && variacion > ϵ
        variacion = valor_best
        for p in P
            valor_p = F(p)
            if valor_p > valor_best
                best = p
                valor_best = valor_p
            end
        end
        variacion = abs(variacion-valor_best)
        Q = sort(P, by=F, rev=true)[1:μ]
        P = copy(Q)
        for q in Q
            for i in 1:(λ/μ)
                push!(P,gaussian_convolution(q, p, σ2, δ))
            end
        end
        iteracion += 1
    end
    return valor_best, best, iteracion, variacion
end

GA_2 (generic function with 1 method)

In [16]:
# test del algoritmo de arriba
F(x) = 5sin(x[1]) + 5cos(x[2]) - 0.1(x[1])^2 - 0.1(x[2])^2 
μ = 5
λ = 25
p = 0.7
σ2 = 0.5
δ = 0.1
n = 2
GA_2(F, μ, λ, p, σ2, δ, n)

(8.717955081023069, [0.852640388675767, 0.19894886727493655], 2, 0.0)

## Algoritmo Genetico

In [23]:
function GA_3(F, popsize, p, σ2, δ, n; max_iter = 10000, ϵ = 1e-8)
    P = [rand(n) for i in 1:popsize]
    best = P[1]
    valor_best = F(best)
    iteracion = 1
    variacion = 1
    while (iteracion <= max_iter) && variacion > ϵ
        variacion = valor_best
        for p in P
            valor_p = F(p)
            if valor_p > valor_best
                best = p
                valor_best = valor_p
            end
        end
        variacion = abs(variacion-valor_best)
        Q = []
        for i in 1:popsize/2
            Pa = rand(P)
            Pb = rand(P)
            child_a, child_b = one_point_crossover(Pa, Pb)
            push!(Q,gaussian_convolution(child_a, p, σ2, δ))
            push!(Q,gaussian_convolution(child_b, p, σ2, δ))
        end
        # one-fifth rule para ir cambiando el σ2
        porcentaje = sum(F(Q[i]) > F(P[i]) for i in 1:popsize)/popsize
        if porcentaje > 0.2
            σ2 = σ2 * 2
        elseif porcentaje < 0.2
            σ2 = σ2 / 1.1
        end
        P = Q
        iteracion += 1
    end
    return valor_best, best, iteracion, variacion
end

GA_3 (generic function with 1 method)

In [39]:
# test del algoritmo de arriba
F(x) = 5sin(x[1]) + 5cos(x[2]) - 0.1(x[1])^2 - 0.1(x[2])^2 
popsize = 30
p = 0.7
σ2 = 0.5
δ = 0.2
n = 2
GA_3(F, popsize, p, σ2, δ, n)

(9.103601775249304, [0.7187798723342739, 0.7602707043649473], 3, 0.0)

## Variantes

Luego las variantes del algoritmo genetico